# **Black Wednesday**

### **1. Import and clean Data**

In [16]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, wilcoxon
from lets_plot import *
LetsPlot.setup_html()

In [17]:
plots = []

UK_data = pd.read_excel("../data/UK.xlsx", skiprows=1).rename(columns={
    "Unnamed: 1": "Month",
    "Economic Activity, Industrial Production, Index": "Industrial_Production",
    "Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate": "National_Exchange_Rate",
    "Exchange Rates, Real Effective Exchange Rate based on Consumer Price Index, Index": "Real_Effective_Exchange_Rate",
    "Prices, Consumer Price Index, All items, Index": "CPI"
}).drop("Unnamed: 0", axis=1)

# The data includes average for the year, not just monthly data so we remove these rows.
UK_data = UK_data[~UK_data['Month'].str.match(r'^\d{4}$')]

# Convert the dates to datetime objects so it is easier to work with.
UK_data['Date'] = pd.to_datetime(UK_data['Month'], format='%b %Y')
UK_data = UK_data.drop("Month", axis=1)

UK_data['Real_Exchange_Rate_Growth'] = UK_data['Real_Effective_Exchange_Rate'].pct_change() * 100
UK_data["CPI_Inflation"] = UK_data["CPI"].pct_change(periods=12) * 100
UK_data = UK_data.dropna()
UK_data["Industrial_Production_Growth"] = UK_data["Industrial_Production"].pct_change(periods=12) * 100  


### **2. Is there evidence that the UK joining the ERM and leaving after Black Wednesday led to changes in the volatility of the monthly growth in the real exchange rate?**

In [18]:
date_joined_ERM = pd.to_datetime("Oct 1990")
date_left_ERM = pd.to_datetime("Sept 1992")

In [19]:
Real_Exchange_Rate_Growth_Plot = (
    ggplot(UK_data, aes(x='Date', y='Real_Exchange_Rate_Growth')) +
    ggtitle("Volatility decreased when they joined") +
    geom_line(color='#1f77b4', size=1.0) +  
    geom_vline(xintercept=date_joined_ERM, color='red', size=0.5) +
    geom_vline(xintercept=date_left_ERM,   color='red', size=0.5) +
    xlab('Date') +
    ylab('Real Effective Exchange Rate growth') +
    theme(
        panel_background=element_rect(fill='white', color='white'),
        panel_grid_major=element_line(color='lightgrey', size=0.5, linetype='dashed'),
        panel_grid_minor=element_line(color='white', size=0.25, linetype='dashed'),
        axis_text=element_text(size=10, color='black'),
        axis_title=element_text(size=12, color='black'),
        plot_title=element_text(size=16, face='bold', hjust=0.5),
        legend_background=element_rect(fill='none')
    ) + 
    ggsize(1000,600)
)
plots.append((Real_Exchange_Rate_Growth_Plot, "Real_Exchange_Rate_Growth_Plot"))

UK_data['Volatility'] = UK_data['Real_Effective_Exchange_Rate'].rolling(window=12).std()


Real_Exchange_Rate_Growth_Volatility_Plot = (
    ggplot(UK_data, aes(x='Date', y='Volatility')) +
    ggtitle("and increased (briefly) when they left") +
    geom_line(color='#1f77b4', size=1.0) +
    geom_vline(xintercept=date_joined_ERM, color='red', size=0.5) +
    geom_vline(xintercept=date_left_ERM, color='red', size=0.5) +
    xlab('Date') +
    ylab('Volatility (12-month Rolling STD)') +
    theme(
        panel_background=element_rect(fill='white', color='white'),
        panel_grid_major=element_line(color='lightgrey', size=0.5, linetype='dashed'),
        panel_grid_minor=element_line(color='white', size=0.25, linetype='dashed'),
        axis_text=element_text(size=10, color='black'),
        axis_title=element_text(size=12, color='black'),
        plot_title=element_text(size=16, face='bold', hjust=0.5),
        legend_background=element_rect(fill='none')
    ) + 
    ggsize(1000,600)
)
plots.append((Real_Exchange_Rate_Growth_Volatility_Plot, "Real_Exchange_Rate_Growth_Volatility_Plot"))
gggrid([Real_Exchange_Rate_Growth_Plot, Real_Exchange_Rate_Growth_Volatility_Plot], ncol=2, widths=[800, 800], fit=True)

### **3. Are there any corresponding changes in the volatility of either the difference in inflation between the UK and Germany or the growth in industrial production?**

In [20]:
German_UK_inflation = pd.DataFrame()
German_UK_inflation["Date"] = UK_data["Date"]
German_UK_inflation["UK_CPI_Inflation"] = UK_data["CPI_Inflation"]

German_CPI = pd.read_excel("../data/GermanyERM.xlsx", skiprows=1).rename(columns={
    "Unnamed: 1": "Month",
    "Economic Activity, Industrial Production, Index": "Industrial_Production",
    "Prices, Consumer Price Index, All items, Index": "CPI"
}).drop("Unnamed: 0", axis=1)

German_CPI = German_CPI[~German_CPI['Month'].str.match(r'^\d{4}$')]
German_CPI['Date'] = pd.to_datetime(German_CPI['Month'], format='%b %Y')
German_CPI = German_CPI.drop("Month", axis=1)

German_CPI["German_CPI_Inflation"] = German_CPI["CPI"].pct_change(periods=12) * 100  
German_CPI = German_CPI.drop("CPI", axis=1)

German_UK_inflation = pd.merge(German_UK_inflation, German_CPI, on="Date")
German_UK_inflation = German_UK_inflation.dropna()

German_UK_inflation["Difference"] = German_UK_inflation["UK_CPI_Inflation"] - German_UK_inflation["German_CPI_Inflation"]

In [21]:
UK_Inflation_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='UK_CPI_Inflation')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('UK CPI Inflation') +
    xlab('Date') +
    ylab('Inflation (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((UK_Inflation_Plot, "UK_Inflation_Plot"))

German_Inflation_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='German_CPI_Inflation')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('German CPI Inflation') +
    xlab('Date') +
    ylab('Inflation (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((German_Inflation_Plot, "German_Inflation_Plot"))

Difference_In_Inflation_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='Difference')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('Difference in Inflation (UK - Germany)') +
    xlab('Date') +
    ylab('Inflation Difference (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((Difference_In_Inflation_Plot, "Difference_In_Inflation_Plot"))

gggrid([Difference_In_Inflation_Plot, UK_Inflation_Plot, German_Inflation_Plot])

In [22]:
UK_Industrial_Growth_Plot = (
    ggplot(UK_data, aes(x='Date', y='Industrial_Production_Growth')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('UK Industrial Production Growth') +
    xlab('Date') +
    ylab('Industrial Production Growth (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)

plots.append((UK_Industrial_Growth_Plot,"UK_Industrial_Growth_Plot"))

UK_Industrial_Growth_Plot

In [23]:

German_UK_inflation["Inflation_Difference_Volatility"] = (
    German_UK_inflation["Difference"]
    .rolling(window=12)
    .std()
)

Inflation_Difference_Volatility_Plot = (
    ggplot(German_UK_inflation, aes(x='Date', y='Inflation_Difference_Volatility')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('Volatility of the Inflation Difference (UK - Germany)') +
    xlab('Date') +
    ylab('Rolling 12-month Std of Inflation Difference') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((Inflation_Difference_Volatility_Plot, "Inflation_Difference_Volatility_Plot"))

UK_data["IP_Growth_Volatility"] = (
    UK_data["Industrial_Production_Growth"]
    .rolling(window=12)
    .std()
)

IP_Growth_Volatility_Plot = (
    ggplot(UK_data, aes(x='Date', y='IP_Growth_Volatility')) +
    geom_line(color='blue', size=1.2) +
    geom_vline(xintercept=date_joined_ERM, color='red', linetype='dashed', size=1) +
    geom_vline(xintercept=date_left_ERM, color='red', linetype='dashed', size=1) +
    ggtitle('Volatility of UK IP Growth') +
    xlab('Date') +
    ylab('Rolling 12-month Std of Inflation Difference') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((IP_Growth_Volatility_Plot, "IP_Growth_Volatility_Plot"))

gggrid([IP_Growth_Volatility_Plot, Inflation_Difference_Volatility_Plot])

### **4.Produce informative figures for the first two sections of the Black Wednesday part of the report**

In [24]:
UK_inflation_plot_before_ERM = (
    ggplot(UK_data[UK_data['Date'] <= date_joined_ERM], aes(x='Date', y='CPI_Inflation')) +
    geom_line(color='blue', size=1.2) +
    ggtitle("UK Inflation was well above 2% during the years preceding the UK's entrance to the ERM") +
    xlab('Date') +
    ylab('Inflation (%)') +
    theme_minimal() +
    theme(plot_title=element_text(face='bold', size=14)) + 
    ggsize(1000,600)
)
plots.append((UK_inflation_plot_before_ERM, "UK_inflation_plot_before_ERM"))

UK_inflation_plot_before_ERM

In [27]:
UK_data['Period'] = np.where(UK_data['Date'] < date_joined_ERM, 'Before ERM',
                             np.where(UK_data['Date'] > date_left_ERM, 'After ERM', 'During ERM'))

# Grouping data by periods
before_erm = UK_data[UK_data['Period'] == 'Before ERM']['Volatility'].dropna()
during_erm = UK_data[UK_data['Period'] == 'During ERM']['Volatility'].dropna()
after_erm = UK_data[UK_data['Period'] == 'After ERM']['Volatility'].dropna()

# Conduct t-tests
before_during_ttest = ttest_ind(before_erm, during_erm, equal_var=False)
during_after_ttest = ttest_ind(during_erm, after_erm, equal_var=False)

# Conduct Wilcoxon tests (non-parametric)
before_during_wilcoxon = wilcoxon(before_erm.iloc[:min(len(before_erm), len(during_erm))],
                                  during_erm.iloc[:min(len(before_erm), len(during_erm))])

# Adjusting Wilcoxon test for differing lengths in "During vs After"
min_length_during_after = min(len(during_erm), len(after_erm))
during_after_wilcoxon = wilcoxon(during_erm.iloc[:min_length_during_after], after_erm.iloc[:min_length_during_after])

# Output results
print("T-Test Results:")
print("Before vs During ERM:", before_during_ttest)
print("During vs After ERM:", during_after_ttest)

print("\nWilcoxon Test Results:")
print("Before vs During ERM:", before_during_wilcoxon)
print("During vs After ERM:", during_after_wilcoxon)

T-Test Results:
Before vs During ERM: TtestResult(statistic=np.float64(3.79753612700855), pvalue=np.float64(0.0006208284650843367), df=np.float64(31.79655425544075))
During vs After ERM: TtestResult(statistic=np.float64(-1.1863748400529912), pvalue=np.float64(0.2431792844191029), df=np.float64(36.30991625771277))

Wilcoxon Test Results:
Before vs During ERM: WilcoxonResult(statistic=np.float64(39.0), pvalue=np.float64(0.0008462667465209961))
During vs After ERM: WilcoxonResult(statistic=np.float64(27.0), pvalue=np.float64(0.00014960765838623047))


In [26]:
for _plot, _plot_name in plots:
    ggsave(plot=_plot, filename=f"{_plot_name}.png", path="../figures")